In [2]:
#Test al features with all fitness functions
from sklearn.model_selection import train_test_split
from keras.src.losses import BinaryCrossentropy
from keras.src.optimizers import Adam
from keras.src.layers import Dense
from keras import Sequential
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [5]:
from Metaheuristicas.fitness_functions import load_and_preprocess_data

X, y = load_and_preprocess_data(filename='Resources/SeisBenchV1_v1_1.json')

In [6]:
X.head()

,f1_t_mean,f2_t_std,f3_t_var,f4_t_entropy,f5_t_kurtosis,f6_t_multiscaleEntropy,f7_t_time2peak,f8_t_rms,f9_t_peak2peak,f10_t_peak2rms,...,f75_w_t_peak2peak_D5,f76_w_t_peak2peak_D6,f77_w_t_peak2rms_A6,f78_w_t_peak2rms_D1,f79_w_t_peak2rms_D2,f80_w_t_peak2rms_D3,f81_w_t_peak2rms_D4,f82_w_t_peak2rms_D5,f83_w_t_peak2rms_D6,f84_w_t_meanEnergyCoeff
0,0.308173,0.205304,0.107536,0.031442,0.229731,0.603239,0.011399,0.205245,0.420602,0.492444,...,0.071505,0.022756,0.343762,0.458053,0.391023,0.369224,0.161757,0.180784,0.898319,0.103404
1,0.322134,0.335965,0.202280,0.170931,0.096625,0.886097,0.118752,0.335950,0.644631,0.331223,...,0.254942,0.045234,0.767478,0.246739,0.116317,0.144180,0.358863,0.223876,0.158059,0.204013
2,0.386335,0.312519,0.183773,0.172402,0.157386,0.672861,0.179005,0.312518,0.855745,0.355332,...,0.551422,0.051260,0.383004,0.139454,0.117581,0.306870,0.199420,0.611760,0.502138,0.183268
3,0.316393,0.895741,0.839779,0.289005,0.021137,0.566134,0.142177,0.895663,0.577603,0.028360,...,0.418491,0.128724,0.106033,0.658328,0.346201,0.250624,0.168671,0.359415,0.408923,0.827045
4,0.324330,0.403592,0.259353,0.088392,0.156549,0.527766,0.039835,0.403528,0.410230,0.270267,...,0.143136,0.022817,0.349813,0.528780,0.415813,0.273371,0.257336,0.234382,0.544004,0.252405


In [5]:
y.head()


0    1
1    0
2    0
3    1
4    1
Name: Type, dtype: int32

In [7]:
import pandas as pd
from sklearn.feature_selection import mutual_info_regression

mi_scores = mutual_info_regression(X, y)

# Crear un dataframe para visualizar mejor los resultados
mi_df = pd.DataFrame({
    'Feature': X.columns,
    'Mutual Information': mi_scores
})

# Calcular el promedio de MI
average_mi = mi_df['Mutual Information'].mean()

# Mostrar los resultados
print(mi_df)
print(f"\nPromedio de Mutual Information: {average_mi}")


                    Feature  Mutual Information
0                 f1_t_mean            0.000000
1                  f2_t_std            0.007686
2                  f3_t_var            0.007135
3              f4_t_entropy            0.020286
4             f5_t_kurtosis            0.000883
..                      ...                 ...
79      f80_w_t_peak2rms_D3            0.015178
80      f81_w_t_peak2rms_D4            0.000000
81      f82_w_t_peak2rms_D5            0.018553
82      f83_w_t_peak2rms_D6            0.002483
83  f84_w_t_meanEnergyCoeff            0.012084

[84 rows x 2 columns]

Promedio de Mutual Information: 0.039426210791181485


In [8]:
from sklearn.feature_selection import chi2

chi2_scores, chi2_pvalues = chi2(X, y)
average_chi2 = chi2_scores.mean()
average_chi2

6.439428677301299

In [12]:
print("Índices de X:", X.index)
print("Índices de y:", y.index)

Índices de X: RangeIndex(start=0, stop=1145, step=1)
Índices de y: Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186],
      dtype='int64', length=1145)


In [13]:
# Reindexar y para que coincida con X
y = y.reset_index(drop=True)

from skrebate import ReliefF

# Inicia ReliefF
relief = ReliefF(n_neighbors=10)  # Ajusta los vecinos si es necesario

# Ajusta ReliefF con X y y
relief.fit(X.values, y.values)

# Calcula el promedio del puntaje de ReliefF
average_relief = relief.feature_importances_.mean()

print(f"Promedio de ReliefF Score: {average_relief}")


Promedio de ReliefF Score: 0.2238900724885615


# Classifiers all features

### Naive Bayes

In [6]:

from sklearn.naive_bayes import GaussianNB

cmodel = GaussianNB()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cmodel.fit(X_train, y_train)
y_pred = cmodel.predict(X_test)
y_pred_proba = cmodel.predict_proba(X_test)[:, 1]  # Probability estimates for AUC

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

In [7]:
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("AUC: ", auc)


Accuracy:  0.9432314410480349
Precision:  0.6086956521739131
Recall:  0.7777777777777778
F1:  0.6829268292682927
AUC:  0.9462875197472354


### Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
y_pred_proba = random_forest.predict_proba(X_test)[:, 1]  # Probability estimates for AUC
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)



In [9]:
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("AUC: ", auc)

Accuracy:  0.9651567944250871
Precision:  0.8
Recall:  0.631578947368421
F1:  0.7058823529411765
AUC:  0.9813432835820897


### Neural Network

In [10]:



RN = Sequential()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
input_dim = X_train.shape[1]
RN.add(Dense(units=64, activation='relu', input_dim=input_dim))
RN.add(Dense(units=32, activation='relu'))
RN.add(Dense(units=1, activation='sigmoid'))
RN.compile(optimizer=Adam(learning_rate=0.001),
           loss=BinaryCrossentropy(),
           metrics=['accuracy'])
history = RN.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)
loss, accuracy = RN.evaluate(X_test, y_test)

print(f'Test Accuracy: {accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred_prob = RN.predict(X_test)  # Predicciones como probabilidades
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir probabilidades a etiquetas (0 o 1)



Epoch 1/100


C:\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4444 - loss: 0.8101 - val_accuracy: 0.9186 - val_loss: 0.3155
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9093 - loss: 0.3082 - val_accuracy: 0.9186 - val_loss: 0.2499
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8990 - loss: 0.2825 - val_accuracy: 0.9186 - val_loss: 0.2242
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8896 - loss: 0.2494 - val_accuracy: 0.9186 - val_loss: 0.1880
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9274 - loss: 0.1611 - val_accuracy: 0.9302 - val_loss: 0.1571
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9613 - loss: 0.1422 - val_accuracy: 0.9302 - val_loss: 0.1307
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9544 - loss: 0.1143 - val_accuracy: 0.9419 - val_loss: 0.1161
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9653 - loss: 0.1000 - val_accuracy: 0.9535 - val_loss: 0.1

In [11]:
# Calcular y mostrar las métricas
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred_prob))  # Usamos probabilidades para AUC


Accuracy: 0.9547038327526133
Precision: 0.6875
Recall: 0.5789473684210527
F1: 0.6285714285714286
AUC: 0.9687745483110762
